# 读写文本格式的数据
anaconda ipython shell  
pandas Input/Output pd.read_csv()  
pandas Input/Output pd.read_table()  
python Built-in Functions open()  
python Built-in Functions list()  
pandas General functions pd.isna()

In [ ]:
import pandas as pd
!cat ch06/ex1.csv

In [ ]:
df=pd.read_csv('ch06/ex1.csv',sep=',')
df

In [ ]:
pd.read_table('ch06/ex1.csv',sep=',')

In [ ]:
!cat ch06/ex2.csv

In [ ]:
pd.read_csv('ch06/ex2.csv',header=None)

In [ ]:
pd.read_csv('ch06/ex2.csv',names=['a','b','c','d','message'])

In [ ]:
names = ['a','b','c','d','message']
pd.read_csv('ch06/ex2.csv',names=names,index_col='message')

In [ ]:
!cat ch06/csv_mindex.csv

In [ ]:
parsed= pd.read_csv('ch06/csv_mindex.csv',index_col=['key1','key2'])
parsed

In [ ]:
list(open('ch06/ex3.txt'))

In [ ]:
result=pd.read_table('ch06/ex3.txt',sep='\s+')
result

In [ ]:
!cat ch06/ex4.csv

In [ ]:
pd.read_csv('ch06/ex4.csv',skiprows=[0,2,3])

In [ ]:
!cat ch06/ex5.csv

In [ ]:
result=pd.read_csv('ch06/ex5.csv')
result

In [ ]:
pd.isna(result)

In [ ]:
result=pd.read_csv('ch06/ex5.csv',na_values=['NULL'])
result

In [ ]:
sentinels={'message':['foo','NA'],'something':['two']}
pd.read_csv('ch06/ex5.csv',na_values=sentinels)

## 逐块读取文本文件

In [ ]:
result = pd.read_csv('ch06/ex6.csv')
result.head()

In [ ]:
pd.read_csv('ch06/ex6.csv',nrows=5)

In [ ]:
chunker=pd.read_csv('ch06/ex6.csv',chunksize=1000)
chunker

In [ ]:
tot =pd.Series([])
for piece in chunker:
    tot = tot.add(piece['key'].value_counts(),fill_value=0)
tot[:10]